## Homework 10

In this homework, you are going to use and compare two different trackers (of your liking) and compare the results.

### Step 1
Decide what video you are going to use for this homework, select an object and generate the template. You can use any video you want (your own, from Youtube, etc.)
and track any object you want (e.g. a car, a pedestrian, etc.).

In [1]:
import cv2
import numpy as np
from IPython.display import HTML

frames, search, x1, y1, x2, y2, width, height, input_video = None, None, None, None, None, None, None, None, None

def init(f = 480, s = 80, x = 317, y = 170, import_file_name = 'dashcam_footage_360p.mp4'):

    global frames, search, x1, y1, x2, y2, width, height, input_video
    
    # Writed frames limit
    frames = f

    # The search limit
    search = s

    # The vehicle`s bounding boxes coords and size
    x1, y1 = x, y,
    x2, y2 = x1 + search, y1 + search
    width = x2 - x1
    height = y2 - y1

    # Import the file
    input_video = cv2.VideoCapture('data/' + import_file_name)

    # Check if video opened successfully
    if input_video.isOpened() == False: 
        print("Error opening video stream or file")
    
def show_video(title, file_name):
    return HTML('<br><h3>' + title + '</h3><video width="320" height="240" controls><source src="data/' + file_name + '" type="video/mp4"></video>')

init()

# Show source video
show_video('Source video', 'dashcam_footage_360p.mp4')

< cv2.VideoCapture 000001ECFADA2EF0>


### Step 2
Initialize a tracker (e.g. KCF).

In [2]:
# Set up tracker
tracker_types = {'MIL':'MIL', 'KCF':'KCF', 'CSRT':'CSRT'}
out_video, tracker, tracker_type = None, None, None

def change_tracker(ttype):
    
    global tracker, out_video, tracker_type
    tracker_type = ttype
    
    if tracker_type == tracker_types['MIL']:
        tracker = cv2.TrackerMIL_create()

    if tracker_type == tracker_types['KCF']:
        tracker = cv2.TrackerKCF_create()

    if tracker_type == tracker_types['CSRT']:
        tracker = cv2.TrackerCSRT_create()

    # Get fitst frame of the video
    ret, frame = input_video.read()

    # Initialize tracker
    bbox = (x1, y1, width, height)
    ok = tracker.init(frame, bbox)

    # Init video writer
    out_video = cv2.VideoWriter('data/result_' + tracker_type + '.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (640, 352))
    
change_tracker(tracker_types['KCF'])

### Step 3-4
Run the tracker on the video and the selected object. Run the tracker for around 10-15 frames.

For each frame, print the bounding box on the image and save it.

In [3]:
def run_tracker():

    frame_index = 0

    # Read until video is completed
    while input_video.isOpened():

        # Capture frame-by-frame
        ret, frame = input_video.read()
        if ret == True and frame_index <= frames:

            ok, bbox = tracker.update(frame)

            # Show the tracker working
            x1, y1 = bbox[0], bbox[1]
            width, height = bbox[2], bbox[3]
            cv2.rectangle(frame, (x1, y1), (x1+width, y1+height), (0, 255, 0), 2)

            # Display the resulting frame
            out_video.write(frame)

            frame_index += 1

        else:
            print('Done!')
            break
    
    # Realease the video capture object
    input_video.release()
    out_video.release()
        
run_tracker()

Done!


In [7]:
# For some reasons result video doesn`t work in Jupyter Notebook, so I fix it via ffmpeg manually (run compress_video_KCF.bat)
show_video('KCF result', 'result_KCF-min.mp4')

### Step 5
Select a different tracker (e.g. CSRT) and repeat steps 2, 3 and 4.


In [5]:
init()
change_tracker('CSRT')
run_tracker()

Done!


In [8]:
# Run compress_video_CSRT.bat before
show_video('CSRT result', 'result_CSRT-min.mp4')

### Step 6
Compare the results:

* Do you see any differences? If so, what are they?

`Sure! The KFC tracker works a bit smoother, but cannot recognize the object when it's comming to the camera.`

`The CSRT tracker gives small jumps but doesn't break at the end of the video.`

* Does one tracker perform better than the other? In what way?

`Yes, the CSRT tracker works better than KFC in this case`

`because the CSRT tracker recognizes resizing, so the tracker doesn't break at the end of the video like with the KFC tracker`